# **Ingestion del archivo camera.json**

##### **01.- Leer archivo JSON usando dataframereader de spark**

In [1]:
#p_file_date = "2025-07-07"
#p_environment = "Development"

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 3, Finished, Available, Finished)

In [2]:
%run configuration

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 4, Finished, Available, Finished)

In [3]:
%run common_functions

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 5, Finished, Available, Finished)

In [4]:
camera_schema = "CameraID INT, CameraResolution STRING, NumberOfEffectivePixel STRING, \
                Aperture STRING, Zoom STRING, RecordableImageFormat STRING, \
                VideoRecording STRING, Flash STRING"

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 6, Finished, Available, Finished)

In [5]:
camera_df = spark.read \
            .schema (camera_schema) \
            .json(f"{bronze_folder_path}/{p_file_date}/camera.json")

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 7, Finished, Available, Finished)

In [6]:
camera_df.printSchema()

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 8, Finished, Available, Finished)

root
 |-- CameraID: integer (nullable = true)
 |-- CameraResolution: string (nullable = true)
 |-- NumberOfEffectivePixel: string (nullable = true)
 |-- Aperture: string (nullable = true)
 |-- Zoom: string (nullable = true)
 |-- RecordableImageFormat: string (nullable = true)
 |-- VideoRecording: string (nullable = true)
 |-- Flash: string (nullable = true)



##### **2.- Eliminar columnas no deseadas del dataframe**

In [7]:
df2 = camera_df.drop("Aperture", "Zoom")

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 9, Finished, Available, Finished)

##### **3.- Cambiar nombres de columnas y añadir ingestion_date y environment**

In [8]:
new_cols = ["camera_id", "camera_resolution", "effective_pixel", "image_format", "video_recording", "flash"]

df3 = df2.toDF(*new_cols)

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 10, Finished, Available, Finished)

In [9]:
from pyspark.sql.functions import lit

df4 = add_ingestion_date(df3) \
        .withColumn("environment", lit(p_environment)) \
         .withColumn("file_date", lit(p_file_date))

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 11, Finished, Available, Finished)

In [10]:
%%sql
DROP TABLE if EXISTS camera

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### **4.- Escribir la salida en delta**

In [11]:
df4.write.format("delta").mode("overwrite").saveAsTable("camera")

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 13, Finished, Available, Finished)

In [12]:
%%sql
SELECT * from camera
limit(10)

StatementMeta(, e6777b1e-99ab-4ae2-bafc-ed75b7278734, 14, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 9 fields>